In [ ]:
import folium
from folium import plugins
import ipywidgets
import geocoder
import geopy
import numpy as np
import pandas as pd
from vega_datasets import data as vds
from pymongo import MongoClient

client = MongoClient('localhost')

db = client['rs_FuSA']

col_archivos_audio = db['archivos_audio']

vmap = folium.Map(location=[-39.825390, -73.231522], zoom_start=12, width=900, height=550, control_scale=True)

r = input("¿Consultar día específico o por catergorías? (d/c): ")

if(r=='d'):
    day = input("Ingrese la fecha a consultar (aaaa-mm-dd): ")
    for i in db.archivos_audio.find({"fecha_grabacion": day},{"duracion":1,"fecha_grabacion":1,"latitud":1,"longitud":1}):
        folium.Marker(
            location=[i['latitud'],i['longitud']],
            popup=str(i['duracion'])+' s\n'+i['fecha_grabacion']).add_to(vmap)

if(r=='c'):
    categories = []
    n = int(input("¿Cuántas categorías desea consultar?: "))
    for i in range(n):
        print("Ingrese categoría ",i+1,": ",sep='',end='')
        categories.append(input())
    ID_list = []
    for i in db.tipos_de_fuentes.find({'tipo':{'$in':categories}}):
        for j in i['archivos']:
            if(j not in ID_list):
                ID_list.append(j)
    for i in db.archivos_audio.find({'ID_archivo':{'$in':ID_list}},{"duracion":1,"fecha_grabacion":1,"latitud":1,"longitud":1}):
        folium.Marker(
            location=[i['latitud'],i['longitud']],
            popup=str(i['duracion'])+' s\n'+i['fecha_grabacion']).add_to(vmap)

vmap